# Quick exporatory Notebook to look at running/processing Fluvial-Particle simulations in a notebook.

In [1]:
import time

from fluvial_particle import Settings
from fluvial_particle import simulate

argdict = {
    "settings_file": "../tests/data/user_options_test.py",
    "output_directory": "../tests/data/output",
    "seed": None,
    "no_postprocess": True,
}
settings_file = argdict["settings_file"]
options = Settings.read(settings_file)
simulate(options, argdict, timer=time.time)


Beginning simulation
Using seed 58336
20
Simulating 20 particles
Particle class: VarSrcParticles
Velocity field will be interpolated from 3D grid
Simulation start time is 0.0, maximum end time is 3600.0, using timesteps of 0.25 (all in seconds).
Remaining time steps 14396/14400 || Elapsed Time: 0:00:00.009149 h:m:s || ETA 0:00:43.904120 h:m:s
Remaining time steps 14392/14400 || Elapsed Time: 0:00:00.018323 h:m:s || ETA 0:00:37.674592 h:m:s
Remaining time steps 14388/14400 || Elapsed Time: 0:00:00.029298 h:m:s || ETA 0:00:38.324535 h:m:s
Remaining time steps 14384/14400 || Elapsed Time: 0:00:00.037823 h:m:s || ETA 0:00:36.271992 h:m:s
Remaining time steps 14380/14400 || Elapsed Time: 0:00:00.048136 h:m:s || ETA 0:00:36.433882 h:m:s
Remaining time steps 14376/14400 || Elapsed Time: 0:00:00.058513 h:m:s || ETA 0:00:36.575518 h:m:s
Remaining time steps 14372/14400 || Elapsed Time: 0:00:00.067541 h:m:s || ETA 0:00:35.954264 h:m:s
Remaining time steps 14368/14400 || Elapsed Time: 0:00:00.078

Remaining time steps 14076/14400 || Elapsed Time: 0:00:00.719861 h:m:s || ETA 0:00:31.372994 h:m:s
Remaining time steps 14072/14400 || Elapsed Time: 0:00:00.731838 h:m:s || ETA 0:00:31.495900 h:m:s
Remaining time steps 14068/14400 || Elapsed Time: 0:00:00.743052 h:m:s || ETA 0:00:31.583098 h:m:s
Remaining time steps 14064/14400 || Elapsed Time: 0:00:00.753466 h:m:s || ETA 0:00:31.634312 h:m:s
Remaining time steps 14060/14400 || Elapsed Time: 0:00:00.764096 h:m:s || ETA 0:00:31.693090 h:m:s
Remaining time steps 14056/14400 || Elapsed Time: 0:00:00.774559 h:m:s || ETA 0:00:31.743361 h:m:s
Remaining time steps 14052/14400 || Elapsed Time: 0:00:00.782821 h:m:s || ETA 0:00:31.703120 h:m:s
Remaining time steps 14048/14400 || Elapsed Time: 0:00:00.791897 h:m:s || ETA 0:00:31.696178 h:m:s
Remaining time steps 14044/14400 || Elapsed Time: 0:00:00.800359 h:m:s || ETA 0:00:31.664898 h:m:s
Remaining time steps 14040/14400 || Elapsed Time: 0:00:00.808979 h:m:s || ETA 0:00:31.640301 h:m:s
Remaining 

Remaining time steps 13744/14400 || Elapsed Time: 0:00:01.456952 h:m:s || ETA 0:00:30.573750 h:m:s
Remaining time steps 13740/14400 || Elapsed Time: 0:00:01.466381 h:m:s || ETA 0:00:30.575942 h:m:s
Remaining time steps 13736/14400 || Elapsed Time: 0:00:01.476370 h:m:s || ETA 0:00:30.589592 h:m:s
Remaining time steps 13732/14400 || Elapsed Time: 0:00:01.485190 h:m:s || ETA 0:00:30.578885 h:m:s
Remaining time steps 13728/14400 || Elapsed Time: 0:00:01.495291 h:m:s || ETA 0:00:30.594416 h:m:s
Remaining time steps 13724/14400 || Elapsed Time: 0:00:01.502957 h:m:s || ETA 0:00:30.560133 h:m:s
Remaining time steps 13720/14400 || Elapsed Time: 0:00:01.511651 h:m:s || ETA 0:00:30.546918 h:m:s
Remaining time steps 13716/14400 || Elapsed Time: 0:00:01.520203 h:m:s || ETA 0:00:30.530931 h:m:s
Remaining time steps 13712/14400 || Elapsed Time: 0:00:01.529090 h:m:s || ETA 0:00:30.521708 h:m:s
Remaining time steps 13708/14400 || Elapsed Time: 0:00:01.538399 h:m:s || ETA 0:00:30.520861 h:m:s
Remaining 

Remaining time steps 13412/14400 || Elapsed Time: 0:00:02.208728 h:m:s || ETA 0:00:30.015870 h:m:s
Remaining time steps 13408/14400 || Elapsed Time: 0:00:02.217642 h:m:s || ETA 0:00:30.006414 h:m:s
Remaining time steps 13404/14400 || Elapsed Time: 0:00:02.227916 h:m:s || ETA 0:00:30.015287 h:m:s
Remaining time steps 13400/14400 || Elapsed Time: 0:00:02.237101 h:m:s || ETA 0:00:30.009395 h:m:s
Remaining time steps 13396/14400 || Elapsed Time: 0:00:02.246957 h:m:s || ETA 0:00:30.012440 h:m:s
Remaining time steps 13392/14400 || Elapsed Time: 0:00:02.256443 h:m:s || ETA 0:00:30.010468 h:m:s
Remaining time steps 13388/14400 || Elapsed Time: 0:00:02.265386 h:m:s || ETA 0:00:30.001241 h:m:s
Remaining time steps 13384/14400 || Elapsed Time: 0:00:02.273395 h:m:s || ETA 0:00:29.979694 h:m:s
Remaining time steps 13380/14400 || Elapsed Time: 0:00:02.283436 h:m:s || ETA 0:00:29.984944 h:m:s
Remaining time steps 13376/14400 || Elapsed Time: 0:00:02.293794 h:m:s || ETA 0:00:29.994221 h:m:s
Remaining 

Remaining time steps 13080/14400 || Elapsed Time: 0:00:02.958430 h:m:s || ETA 0:00:29.339821 h:m:s
Remaining time steps 13076/14400 || Elapsed Time: 0:00:02.966250 h:m:s || ETA 0:00:29.319464 h:m:s
Remaining time steps 13072/14400 || Elapsed Time: 0:00:02.975167 h:m:s || ETA 0:00:29.309987 h:m:s
Remaining time steps 13068/14400 || Elapsed Time: 0:00:02.982693 h:m:s || ETA 0:00:29.286863 h:m:s
Remaining time steps 13064/14400 || Elapsed Time: 0:00:02.991776 h:m:s || ETA 0:00:29.279066 h:m:s
Remaining time steps 13060/14400 || Elapsed Time: 0:00:02.999317 h:m:s || ETA 0:00:29.256218 h:m:s
Remaining time steps 13056/14400 || Elapsed Time: 0:00:03.007740 h:m:s || ETA 0:00:29.242039 h:m:s
Remaining time steps 13052/14400 || Elapsed Time: 0:00:03.016330 h:m:s || ETA 0:00:29.229512 h:m:s
Remaining time steps 13048/14400 || Elapsed Time: 0:00:03.027358 h:m:s || ETA 0:00:29.240559 h:m:s
Remaining time steps 13044/14400 || Elapsed Time: 0:00:03.035200 h:m:s || ETA 0:00:29.220797 h:m:s
Remaining 

...

Remaining time steps 10424/14400 || Elapsed Time: 0:00:09.015487 h:m:s || ETA 0:00:23.644390 h:m:s
Remaining time steps 10420/14400 || Elapsed Time: 0:00:09.024260 h:m:s || ETA 0:00:23.634535 h:m:s
Remaining time steps 10416/14400 || Elapsed Time: 0:00:09.032343 h:m:s || ETA 0:00:23.622877 h:m:s
Remaining time steps 10412/14400 || Elapsed Time: 0:00:09.042084 h:m:s || ETA 0:00:23.615557 h:m:s
Remaining time steps 10408/14400 || Elapsed Time: 0:00:09.050329 h:m:s || ETA 0:00:23.604330 h:m:s
Remaining time steps 10404/14400 || Elapsed Time: 0:00:09.058629 h:m:s || ETA 0:00:23.593249 h:m:s
Remaining time steps 10400/14400 || Elapsed Time: 0:00:09.066791 h:m:s || ETA 0:00:23.581818 h:m:s
Remaining time steps 10396/14400 || Elapsed Time: 0:00:09.074897 h:m:s || ETA 0:00:23.570249 h:m:s
Remaining time steps 10392/14400 || Elapsed Time: 0:00:09.082681 h:m:s || ETA 0:00:23.557850 h:m:s
Remaining time steps 10388/14400 || Elapsed Time: 0:00:09.091596 h:m:s || ETA 0:00:23.548391 h:m:s
Remaining 

Remaining time steps 10092/14400 || Elapsed Time: 0:00:09.731168 h:m:s || ETA 0:00:22.803965 h:m:s
Remaining time steps 10088/14400 || Elapsed Time: 0:00:09.739485 h:m:s || ETA 0:00:22.793242 h:m:s
Remaining time steps 10084/14400 || Elapsed Time: 0:00:09.747222 h:m:s || ETA 0:00:22.781166 h:m:s
Remaining time steps 10080/14400 || Elapsed Time: 0:00:09.756076 h:m:s || ETA 0:00:22.771706 h:m:s
Remaining time steps 10076/14400 || Elapsed Time: 0:00:09.763969 h:m:s || ETA 0:00:22.760008 h:m:s
Remaining time steps 10072/14400 || Elapsed Time: 0:00:09.772650 h:m:s || ETA 0:00:22.750152 h:m:s
Remaining time steps 10068/14400 || Elapsed Time: 0:00:09.780984 h:m:s || ETA 0:00:22.739489 h:m:s
Remaining time steps 10064/14400 || Elapsed Time: 0:00:09.789788 h:m:s || ETA 0:00:22.729922 h:m:s
Remaining time steps 10060/14400 || Elapsed Time: 0:00:09.797954 h:m:s || ETA 0:00:22.718878 h:m:s
Remaining time steps 10056/14400 || Elapsed Time: 0:00:09.806314 h:m:s || ETA 0:00:22.708290 h:m:s
Remaining 

Remaining time steps 9756/14400 || Elapsed Time: 0:00:10.434554 h:m:s || ETA 0:00:21.927621 h:m:s
Remaining time steps 9752/14400 || Elapsed Time: 0:00:10.443017 h:m:s || ETA 0:00:21.917527 h:m:s
Remaining time steps 9748/14400 || Elapsed Time: 0:00:10.451332 h:m:s || ETA 0:00:21.907124 h:m:s
Remaining time steps 9744/14400 || Elapsed Time: 0:00:10.459598 h:m:s || ETA 0:00:21.896624 h:m:s
Remaining time steps 9740/14400 || Elapsed Time: 0:00:10.466884 h:m:s || ETA 0:00:21.884077 h:m:s
Remaining time steps 9736/14400 || Elapsed Time: 0:00:10.475220 h:m:s || ETA 0:00:21.873734 h:m:s
Remaining time steps 9732/14400 || Elapsed Time: 0:00:10.483132 h:m:s || ETA 0:00:21.862508 h:m:s
Remaining time steps 9728/14400 || Elapsed Time: 0:00:10.492185 h:m:s || ETA 0:00:21.853664 h:m:s
Remaining time steps 9724/14400 || Elapsed Time: 0:00:10.499284 h:m:s || ETA 0:00:21.840757 h:m:s
Remaining time steps 9720/14400 || Elapsed Time: 0:00:10.509262 h:m:s || ETA 0:00:21.833840 h:m:s
Remaining time steps

Remaining time steps 9420/14400 || Elapsed Time: 0:00:11.130147 h:m:s || ETA 0:00:21.059874 h:m:s
Remaining time steps 9416/14400 || Elapsed Time: 0:00:11.139243 h:m:s || ETA 0:00:21.051224 h:m:s
Remaining time steps 9412/14400 || Elapsed Time: 0:00:11.147916 h:m:s || ETA 0:00:21.041776 h:m:s
Remaining time steps 9408/14400 || Elapsed Time: 0:00:11.156575 h:m:s || ETA 0:00:21.032302 h:m:s
Remaining time steps 9404/14400 || Elapsed Time: 0:00:11.164433 h:m:s || ETA 0:00:21.021319 h:m:s
Remaining time steps 9400/14400 || Elapsed Time: 0:00:11.172762 h:m:s || ETA 0:00:21.011229 h:m:s
Remaining time steps 9396/14400 || Elapsed Time: 0:00:11.180470 h:m:s || ETA 0:00:20.999976 h:m:s
Remaining time steps 9392/14400 || Elapsed Time: 0:00:11.188934 h:m:s || ETA 0:00:20.990145 h:m:s
Remaining time steps 9388/14400 || Elapsed Time: 0:00:11.196591 h:m:s || ETA 0:00:20.978806 h:m:s
Remaining time steps 9384/14400 || Elapsed Time: 0:00:11.205713 h:m:s || ETA 0:00:20.970213 h:m:s
Remaining time steps

Remaining time steps 9084/14400 || Elapsed Time: 0:00:11.830420 h:m:s || ETA 0:00:20.221894 h:m:s
Remaining time steps 9080/14400 || Elapsed Time: 0:00:11.838478 h:m:s || ETA 0:00:20.211546 h:m:s
Remaining time steps 9076/14400 || Elapsed Time: 0:00:11.846500 h:m:s || ETA 0:00:20.201143 h:m:s
Remaining time steps 9072/14400 || Elapsed Time: 0:00:11.855395 h:m:s || ETA 0:00:20.192228 h:m:s
Remaining time steps 9068/14400 || Elapsed Time: 0:00:11.864020 h:m:s || ETA 0:00:20.182855 h:m:s
Remaining time steps 9064/14400 || Elapsed Time: 0:00:11.872043 h:m:s || ETA 0:00:20.172459 h:m:s
Remaining time steps 9060/14400 || Elapsed Time: 0:00:11.880527 h:m:s || ETA 0:00:20.162851 h:m:s
Remaining time steps 9056/14400 || Elapsed Time: 0:00:11.888804 h:m:s || ETA 0:00:20.152892 h:m:s
Remaining time steps 9052/14400 || Elapsed Time: 0:00:11.897124 h:m:s || ETA 0:00:20.143008 h:m:s
Remaining time steps 9048/14400 || Elapsed Time: 0:00:11.905117 h:m:s || ETA 0:00:20.132574 h:m:s
Remaining time steps

In [2]:
from threading import Thread
import pyvista as pv
import pyvistaqt as pvqt
import meshio
import vtk
import numpy as np
import h5py
# pv.set_jupyter_backend('panel')

# pyvistaqt Crashes
* Testing pyvistaqt crashes kernal
* un-comment cell below to see

In [3]:
# import pyvista as pv
# from pyvistaqt import BackgroundPlotter

# sphere = pv.Sphere()

# plotter = BackgroundPlotter()
# plotter.add_mesh(sphere)
# plotter.sho()

# # can now operate on the sphere and have it updated in the background
# # sphere.points *= 0.5

# XDMF
* pyvista uses meshio to read xdmf files.  However, meshio can not read our .xmf files (see cell below).
* comment in meshio issues suggest xdmf is not well supported, don't know how valid the statement is: https://github.com/nschloe/meshio/issues/1273#issue-1117218893
* Interestingly paraview has an XDMF reader that works!

In [4]:
meshio.read('../tests/data/output/particles.xmf')

Unknown section 'Grid'.


Error: Couldn't read file ../tests/data/output/particles.xmf as xdmf

SystemExit: 1

/home/rmcd/miniconda3/envs/fluvial-particle/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Particle helper Functions
* pretty straight-forward to write our own particle helper functions

In [6]:
def get_h5file(filename: str) -> h5py._hl.files.File:
    return h5py.File(filename)

def get_num_timesteps(f: h5py._hl.files.File) -> int:
    return f["coordinates"].get("x").shape[0]

def get_points(f: h5py._hl.files.File, time: int, twod: bool=False) -> np.ndarray:
    # f = h5py.File(filename)
    pt_dim = f["coordinates"].get("x").shape[1]
    x = f["coordinates"].get("x")
    y = f["coordinates"].get("y")
    z = f["coordinates"].get("z")
    if not twod:
        return np.array([[[x[i,j], y[i,j], z[i,j]] for i in [time] for j in np.arange(pt_dim)]][0])
    else:
        # returns z as 0.5 so it sits above 2d mesh
        return np.array([[[x[i,j], y[i,j], 0.5] for i in [time] for j in np.arange(pt_dim)]][0])

# Example files
* particle.h5 file generated by running code in fist code block above

In [7]:
my_mesh = pv.read('../tests/data/Result_FM_MEander_1_long_2D1.vtk')
file = get_h5file('../tests/data/output/particles.h5')


# Inline animation
* https://panel.holoviz.org/reference/panes/VTK.html#panes-gallery-vtk

In [8]:
import panel as pn
pn.extension('vtk')
pl = pv.Plotter()
pl.add_mesh(my_mesh, scalars="IBC")
points = get_points(file, 0, twod=True)
point_cloud = pv.PolyData(points)
pl.add_mesh(point_cloud, point_size=3)
sim_pan = pn.panel(pl.ren_win, width=1000, orientation_widget=True)
nts = get_num_timesteps(file)
print(nts)
player = pn.widgets.Player(name='Player', start=0, end=nts-1, loop_policy='loop', interval=nts-1)
@pn.depends(value=player.param.value)
def newpts(value):
    print(value)
    points = get_points(file, value, twod=True)
    point_cloud.overwrite(pv.PolyData(points))
#     time.sleep(0.5)
    sim_pan.synchronize()
    
pn.Column(sim_pan, player, newpts).servable()
# sim_pan

3601
0


Column
    [0] VTKRenderWindowSynchronized(vtkOSOpenGLRenderWindow, color_mappers=[LinearColorMapper(id='100...], orientation_widget=True, width=1000)
    [1] Player(end=3600, interval=3600, loop_policy='loop')
    [2] ParamFunction(function)

In [9]:
pl.camera_position

[(223.21642184257507, 0.4330129623413086, 882.7372716587969),
 (223.21642184257507, 0.4330129623413086, 0.25),
 (0.0, 1.0, 0.0)]

# Create animation .mp4
* https://docs.pyvista.org/examples/02-plot/movie.html#sphx-glr-examples-02-plot-movie-py

In [13]:
plotter = pv.Plotter()
plotter.open_movie("../tests/data/ouput/test.mp4")
# plotter.view_xy()
plotter.add_mesh(my_mesh, scalars="IBC")
points = get_points(file, 0, twod=True)
point_cloud = pv.PolyData(points)
plotter.add_mesh(point_cloud, point_size=3, cmap='r' )

# plotter.camera_postion = [(732.7206856912, 509.9372768109662, 509.7542638486249),
#  (223.21642184257507, 0.4330129623413086, 0.25),
#  (0.0, 0.0, 1.0)]

# plotter.show(auto_close=False)
plotter.write_frame()

def newpts(index):
    points = get_points(file, index, twod=True)
    point_cloud.overwrite(pv.PolyData(points))
#     plotter.render()
    plotter.write_frame()
    

for i in range(3600):
    newpts(i)
    print(i)
    
plotter.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
